# Mathematical model kinetics

In [2]:
#import warnings
#warnings.filterwarnings('ignore')
%matplotlib inline
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [3]:
import pandas as pd
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)
pd.set_option('display.width', 10000)
import numpy as np
import yaml

from vessel_manoeuvring_models.visualization.plot import track_plot, plot
from phd.visualization.plot_ship import track_plots
from vessel_manoeuvring_models.prime_system import PrimeSystem
from phd.visualization.plot_prediction import plot_total_force, plot_force_components, predict, plot_compare_model_forces, plot_parameter_contributions
from phd.visualization.bokeh_plotting import create_tab
from vessel_manoeuvring_models.symbols import *
import matplotlib.pyplot as plt
from vessel_manoeuvring_models.substitute_dynamic_symbols import run, lambdify, remove_functions, prime
from vessel_manoeuvring_models.parameters import df_parameters
p = df_parameters["symbol"]
import statsmodels.api as sm
import sympy as sp
from sympy import Eq, symbols, latex
import paper
from paper import save_fig, save_eq
import vessel_manoeuvring_models

Loading BokehJS ...

In [4]:
#%reload_kedro
ship='wPCC'
ship_data = catalog.load(f"{ship}.ship_data")
model_loaders = catalog.load(f"{ship}.models_VCT")
models_rudder_VCT_loaders = catalog.load(f"{ship}.models_rudder_VCT")

[02/09/24 11:32:27] INFO     Loading data from 'wPCC.ship_data' (YAMLDataSet)...                ]8;id=656918;file:///home/maa/miniconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=925086;file:///home/maa/miniconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

[02/09/24 11:32:28] WARNING  /home/maa/miniconda3/envs/phd/lib/python3.10/site-packages/kedro/io/pa ]8;id=120659;file:///home/maa/miniconda3/envs/phd/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=722000;file:///home/maa/miniconda3/envs/phd/lib/python3.10/warnings.py#109\109]8;;\
                             rtitioned_dataset.py:200: KedroDeprecationWarning:                                    
                             'PartitionedDataset' has been moved to `kedro-datasets` and will be                   
                             removed in Kedro 0.19.0.                                                              
                               warnings.warn(                                                                      
                                                                                                                   

                    WARNING  /home/maa/miniconda3/envs/phd/lib/python3.10/site-packages/kedro/io/__ ]8;id=831742;file:///home/maa/miniconda3/envs/phd/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=296830;file:///home/maa/miniconda3/envs/phd/lib/python3.10/warnings.py#109\109]8;;\
                             init__.py:44: KedroDeprecationWarning: 'AbstractDataSet' has been                     
                             renamed to 'AbstractDataset', and the alias will be removed in Kedro                  
                             0.19.0                                                                                
                               return getattr(kedro.io.core, name)                                                 
                                                                                                                   

                    INFO     Loading data from 'wPCC.models_VCT' (PartitionedDataset)...        ]8;id=836717;file:///home/maa/miniconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=743079;file:///home/maa/miniconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

                    INFO     Loading data from 'wPCC.models_rudder_VCT' (PartitionedDataset)... ]8;id=185284;file:///home/maa/miniconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=327223;file:///home/maa/miniconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

In [5]:
model_loaders


{
    'semiempirical_covered': <bound method AbstractDataset.load of <phd.extras.datasets.model_dataset.ModularVesselSimulatorDataSet object at 0x7f0862321de0>>
}

In [6]:
models_rudder_VCT_loaders


{
    'Abkowitz': <bound method AbstractDataset.load of <phd.extras.datasets.model_dataset.ModularVesselSimulatorDataSet object at 0x7f0862321d20>>
}

In [7]:
model = model_loaders['semiempirical_covered']()

In [8]:
model_abkowitz = models_rudder_VCT_loaders['Abkowitz']()

In [9]:
eq_X_D = remove_functions(model.X_D_eq)

In [10]:
eq_X = sp.Eq(model.X_eq.lhs,model.X_eq.rhs.subs(model.X_D_eq.rhs,X_D_))
eq_Y = sp.Eq(model.Y_eq.lhs,model.Y_eq.rhs.subs(model.Y_D_eq.rhs,Y_D_))
eq_N = sp.Eq(model.N_eq.lhs,model.N_eq.rhs.subs(model.N_D_eq.rhs,N_D_))

save_eq(remove_functions(eq_X), file_name='X')
save_eq(remove_functions(eq_Y), file_name='Y')
save_eq(remove_functions(eq_N), file_name='N')

<IPython.core.display.HTML object>

<IPython.core.display.Math object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.Math object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.Math object>

<IPython.core.display.HTML object>

In [11]:
X_D_eq = remove_functions(model.X_D_eq).subs([
    ("X_W",0),
    ("X_RHI",0),
])
Y_D_eq = remove_functions(model.Y_D_eq).subs("Y_W",0)
N_D_eq = remove_functions(model.N_D_eq).subs("N_W",0)

save_eq(remove_functions(X_D_eq), file_name='X_D')
save_eq(remove_functions(Y_D_eq), file_name='Y_D')
save_eq(remove_functions(N_D_eq), file_name='N_D')

<IPython.core.display.HTML object>

<IPython.core.display.Math object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.Math object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.Math object>

<IPython.core.display.HTML object>

In [12]:
model.show_subsystems()

hull: PrimeEquationSubSystem
propellers: PropellersSimpleSystem
rudder_port: SemiempiricalRudderSystemCovered
rudder_stbd: SemiempiricalRudderSystemCovered
rudders: Rudders
rudder_hull_interaction: RudderHullInteractionSystem
wind_force: DummyWindForceSystem


## Hull

In [13]:
from phd.paper.equation import to_latex
from IPython.display import display, Latex, Math, HTML

eq_latex = to_latex(model.subsystems['hull'].equations['Y_H'])

eq_latex = eq_latex.replace(r'Y_{rrr} r^{3}',r'\cancel{Y_{rrr}} r^{3}')

In [14]:
def cancel(eq:sp.Eq,cancels = []):
    subs={}
    for cancel in cancels:
        subs[cancel]=sp.Symbol(fr'\cancel{{{cancel}}}')
    eq = eq.subs(subs)
    return eq

In [15]:
cancels = [
    p.Yrrr,p.Yvrr,p.Yvvr,p.Yvvv,
    p.Nrrr,p.Nvrr,p.Nvvr,p.Nvvv,
]
for name, eq in model.subsystems['hull'].equations.items():
    #save_eq(cancel(prime(eq)), file_name=name)
    save_eq(prime(cancel(eq, cancels=cancels)), file_name=name)

<IPython.core.display.HTML object>

<IPython.core.display.Math object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.Math object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.Math object>

<IPython.core.display.HTML object>

## Propellers

In [16]:
for name, eq in model.subsystems['propellers'].equations.items():
        
    if isinstance(model.subsystems['propellers'], vessel_manoeuvring_models.models.subsystem.PrimeEquationSubSystem):
        save_eq(prime(eq), file_name=name)
    else:
        save_eq(eq, file_name=name)

<IPython.core.display.HTML object>

<IPython.core.display.Math object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.Math object>

<IPython.core.display.HTML object>

## Rudder hull interaction

In [17]:
for name, eq in model.subsystems['rudder_hull_interaction'].equations.items():
    if isinstance(model.subsystems['rudder_hull_interaction'], vessel_manoeuvring_models.models.subsystem.PrimeEquationSubSystem):
        save_eq(prime(eq), file_name=name)
    else:
        save_eq(eq, file_name=name)

<IPython.core.display.HTML object>

<IPython.core.display.Math object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.Math object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.Math object>

<IPython.core.display.HTML object>

## Pure mathematical rudder model

In [18]:
model_abkowitz.show_subsystems()

hull: PrimeEquationSubSystem
propellers: PropellersSimpleSystem
rudders: AbkowitzRudderSystem
rudder_hull_interaction: RudderHullInteractionDummySystem
wind_force: DummyWindForceSystem


In [19]:
str(p.Ndelta)

'N_{delta}'

In [20]:
subs = {sp.Symbol(str(row['symbol'])):sp.Symbol(str(row['symbol']).replace('delta',r'\delta')) for name, row in df_parameters.iterrows()}
for name, eq in model_abkowitz.subsystems['rudders'].equations.items():
    eq = eq.subs(delta,sp.Symbol(r'\delta'))
    model_abkowitz.subsystems['rudders'].equations[name] = eq.subs(subs)
    

In [21]:
for name, eq in model_abkowitz.subsystems['rudders'].equations.items():
    
    file_name = f"{name}_math"
    
    if isinstance(model_abkowitz.subsystems['rudders'], vessel_manoeuvring_models.models.subsystem.PrimeEquationSubSystem):
        save_eq(prime(eq), file_name=file_name)
    else:
        save_eq(eq, file_name=file_name)

<IPython.core.display.HTML object>

<IPython.core.display.Math object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.Math object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.Math object>

<IPython.core.display.HTML object>

## Semi-empirical rudder model

In [22]:
rudder = model.subsystems['rudder_port']
#rudder.equations['r_p_port'] = rudder.equations['r_p_port'].subs('r_p_port','r_x_port')

In [23]:
rudder.equations['X_R_port']

Eq(X_R_port, -D_R_port*cos(alpha_f_port) + L_R_port*sin(alpha_f_port))

In [24]:
keys = list(rudder.equations.keys())
keys.reverse()

### Remove stall

In [25]:
rudder.equations['C_L_port'] = sp.Eq(rudder.equations['C_L_port'].lhs,rudder.equations['C_L_port'].rhs.args[0][0])

In [26]:
rudder.equations['C_L_port']

Eq(C_L_port, lambda_gap_port*(alpha_port*dC_L_dalpha_port + C_D_crossflow_port*alpha_port*Abs(alpha_port)/AR_e_port))

In [27]:
C_D_crossflow_port = symbols("C_D_crossflow_port")  # crossflow drag coefficient
delta = symbols("delta")  # crossflow drag coefficient
subs={
    C_D_crossflow_port : symbols("C_DC"),
    
}
for name,eq in rudder.equations.items():
    rudder.equations[name] = eq.subs(subs)


In [28]:
rudder.equations["C_L_port"]

Eq(C_L_port, lambda_gap_port*(alpha_port*dC_L_dalpha_port + C_DC*alpha_port*Abs(alpha_port)/AR_e_port))

In [29]:
replace_latex = {
    r"dC_{L dalpha }" : r"\frac{\partial C_L}{\partial \alpha}",
}
for name in keys:
    eq = rudder.equations[name]
    name_clean = name.replace('_port','')
    file_name = f"{name_clean}_semiempirical"    
    save_eq(eq, file_name=file_name, subs={'port':''}, replace_latex=replace_latex)

<IPython.core.display.HTML object>

<IPython.core.display.Math object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.Math object>

<IPython.core.display.HTML object>

In [30]:
s_latex = ""
for name in keys:
    eq = rudder.equations[name]
    name_clean = name.replace('_port','')
    file_name = f"{name_clean}_semiempirical"
    
    eq_latex = fr"""
\begin{{equation}}
\label{{eq:{file_name}}}
\input{{equations/mathematical_model_kinetics.{file_name}}}
\end{{equation}}
%"""
    s_latex+=eq_latex

print(s_latex)


\begin{equation}
\label{eq:N_R_semiempirical}
\input{equations/mathematical_model_kinetics.N_R_semiempirical}
\end{equation}
%
\begin{equation}
\label{eq:Y_R_semiempirical}
\input{equations/mathematical_model_kinetics.Y_R_semiempirical}
\end{equation}
%
\begin{equation}
\label{eq:X_R_semiempirical}
\input{equations/mathematical_model_kinetics.X_R_semiempirical}
\end{equation}
%
\begin{equation}
\label{eq:L_R_semiempirical}
\input{equations/mathematical_model_kinetics.L_R_semiempirical}
\end{equation}
%
\begin{equation}
\label{eq:L_R_U_semiempirical}
\input{equations/mathematical_model_kinetics.L_R_U_semiempirical}
\end{equation}
%
\begin{equation}
\label{eq:L_R_C_semiempirical}
\input{equations/mathematical_model_kinetics.L_R_C_semiempirical}
\end{equation}
%
\begin{equation}
\label{eq:lambda_R_semiempirical}
\input{equations/mathematical_model_kinetics.lambda_R_semiempirical}
\end{equation}
%
\begin{equation}
\label{eq:f_semiempirical}
\input{equations/mathematical_model_kinetics.f_s

In [31]:
model.Fn0

0.02

In [32]:
import re
new = r"\\delta"
old = r"delta"
string = r"delta"
Math(re.sub(fr"\W*{old}", new, string))

<IPython.core.display.Math object>

In [33]:
Math(string)

<IPython.core.display.Math object>

In [34]:
re.search(r"\w+(delta)",r"Ndelta")

<re.Match object; span=(0, 6), match='Ndelta'>

In [35]:
re.search(r"\w+(delta)",r"\delta")

In [36]:
re.sub(r"\w+(delta)",r"delta", r"Ndelta")

'delta'